In [3]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path as pt
import matplotlib.gridspec as gridspec
from ipywidgets import widgets

In [10]:
def gauss_func(x, A, sig, mu): return A*np.exp(-1/2 * ((x-mu)/sig)**2)

In [147]:
def plot_simulation(freq_range, original_signal, corr_collection, sim_range):
    %matplotlib widget
    plt.close("all")
    fig = plt.figure(tight_layout=True, figsize=(12, 7))
    gs = gridspec.GridSpec(2, 2)

    ax = fig.add_subplot(gs[0, 0])
    ax1 = fig.add_subplot(gs[0, 1])
    ax2 = fig.add_subplot(gs[1, :])
    
    ax.plot(freq_range, original_signal.T,  ".")
    ax.legend(["Res OFF", "Res ON"])
    
    res_off_background, res_on_signal = original_signal
    original_signal_depletion = (1 - (res_on_signal/res_off_background))*100
    
    ax1.plot(freq_range, original_signal_depletion,  ".k")
    ax1.legend([f"Signal (%)"])
    
    ax2.plot(sim_range, corr_collection, ".-")
    
    corr_collection = corr_collection.T
    corr_diff = corr_collection[0] - corr_collection[1]
    
    ax2.legend([f"Background corr. {corr_collection[0].mean():.2f}({corr_collection[0].std():.2f})",\
                f"Signal corr. {corr_collection[1].mean():.2f}({corr_collection[1].std():.2f})"],\
              title=f"Corr. diff: {corr_diff.mean():.2f}({corr_diff.std():.2f})")
    
    ax.set(title="Simulating Experimental data", xlabel="Frequency (MHz)", ylabel="Counts")
    ax1.set(title="Simulating Experimental data", xlabel="Frequency (MHz)", ylabel="Depletion (%)")
    ax2.set(xlabel="Simulation No. (#)", ylabel="Correlation factor (r)")
    
    for _ in (ax, ax1, ax2):
        _.minorticks_on()
        _.grid()
    
def get_lineshape(base_counts, mu, freq_range, fwhm, A):
    
    fwhm = fwhm*1e-3 # FWHM in MHz
    sig = fwhm/(2*np.sqrt(2*np.log(2))) # Sigma in MHz
    A = -(A/100)*base_counts # Intensity
    
    signal_shape = gauss_func(freq_range, A, sig, mu)
    
    return signal_shape


In [148]:
@widgets.interact(A=10, fwhm=500, mu=3, noise_sig=50, extra_noise_sig=2, sim_range=5, step_size=10)
def simulate_correlation(A, fwhm, mu, noise_sig, extra_noise_sig, sim_range, step_size, plot=True):

    # Defining frequency range
    start_freq = 1 # MHz
    end_freq = 5 # MHz
    step_size = 10*1e-3 # KHz

    freq_range = np.arange(start_freq, end_freq, step_size)

    # Background counts with noise for OFF resonance
    base_counts = 1000
    res_off_background = np.random.normal(base_counts, noise_sig, freq_range.shape)

    # Calculating signal shape (Gaussian distribution)
    signal_shape = get_lineshape(base_counts, mu, freq_range, fwhm, A)
    
    # Background counts with noise for ON resonance
    extra_noise_sig = noise_sig/2
    res_on_background = res_off_background + np.random.normal(0, extra_noise_sig, freq_range.shape)
    res_on_signal = signal_shape + res_on_background

    original_signal = np.array([res_off_background, res_on_signal])

    # Calculating correlation factor by running several number of simulations
    corr_collection = []
    sim_range = np.arange(sim_range)

    for _ in sim_range:

        res_off_background = np.random.normal(base_counts, noise_sig, freq_range.shape)
        
        res_on_background = res_off_background + np.random.normal(0, extra_noise_sig, freq_range.shape)
        res_on_signal = signal_shape + res_on_background

        correlation_noise = np.corrcoef(res_off_background, res_on_background)[0, 1]
        correlation_sig = np.corrcoef(res_off_background, res_on_signal)[0, 1]
        
        corr_collection.append([correlation_noise, correlation_sig])

    corr_collection = np.array(corr_collection)
    
    if plot: plot_simulation(freq_range, original_signal, corr_collection, sim_range)
    else: 
        return freq_range, corr_collection

interactive(children=(IntSlider(value=10, description='A', max=30, min=-10), IntSlider(value=500, description=…

In [159]:
@widgets.interact(A=10, fwhm=500, mu=3, noise_sig=50, extra_noise_sig=2, sim_range="5", step_size=10, _from="1", _to="10", _step="1", variable="fwhm")
def simulate_correlation_fwhm(A, fwhm, mu, noise_sig, extra_noise_sig, sim_range, step_size, _from="1", _to="10", _step="1", variable="fwhm"):
    
    variable_list = {
        "A":A, "fwhm":fwhm, "noise_sig":noise_sig, "step_size":step_size, "mu":mu,
        "extra_noise_sig":extra_noise_sig, "sim_range":int(sim_range), "plot":False
    }
    
    variable_range = np.arange(int(_from), int(_to), float(_step))
    
    datas = []
    
    for _ in variable_range:
        variable_list[variable] = _
        temp = simulate_correlation(**variable_list)
        datas.append(temp)

    datas = np.array(datas).T
    freq_range, corr_collection = datas
    
    corr_diff_collection = []
    for corr in corr_collection:
        corr = corr.T
        corr_diff = corr[0] - corr[1]
        corr_diff = corr_diff.mean()
        corr_diff_collection.append(corr_diff)
    
    corr_diff_collection = np.array(corr_diff_collection)
    
    %matplotlib widget
    
    fig, ax = plt.subplots()
    ax.plot(variable_range.T, corr_diff_collection.T, ".-")
    
    ax.set(title=f"Correlation difference as a function of {variable}", xlabel=variable, ylabel="Correlation factor difference from background")

interactive(children=(IntSlider(value=10, description='A', max=30, min=-10), IntSlider(value=500, description=…

In [65]:
%matplotlib widget
noise_mu = 1000
noise_sig = 5
num = 100
res_off_noise = np.random.normal(noise_mu, noise_sig, num)
res_on_noise = res_off_noise + np.random.normal(0, noise_sig/2, num)
plt.plot(res_off_noise, ".-")
plt.plot(res_on_noise, ".-")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
%matplotlib widget

res_off_noise = np.random.normal(0, 0, 100)
plt.plot(res_off_noise, ".-")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [105]:
0.11*9

0.99